### Loading of all the SQL tables using sqlite3 (to run locally)

In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("ecommerce.db")
cursor = conn.cursor()
cursor.execute("PRAGMA foreign_keys = ON;")

In [ ]:
'''cursor.execute("""DROP TABLE IF EXISTS distribution_centers;""")
cursor.execute("""DROP TABLE IF EXISTS events;""")
cursor.execute("""DROP TABLE IF EXISTS inventory_items;""")
cursor.execute("""DROP TABLE IF EXISTS order_items;""")
cursor.execute("""DROP TABLE IF EXISTS orders;""")
cursor.execute("""DROP TABLE IF EXISTS products;""")
cursor.execute("""DROP TABLE IF EXISTS users;""")

conn.commit()'''

In [ ]:
'''import os

conn.close()
if os.path.exists("ecommerce.db"):
    os.remove("ecommerce.db")
    print("Database file deleted. You now have a clean slate.")'''

Database file deleted. You now have a clean slate.


Defining the correct schemas(types, primary and foreign keys) as sqlite doesn't do it automatically and they aren't defined in the csv files

In [19]:
cursor.execute("""CREATE TABLE distribution_centers (
    id INTEGER PRIMARY KEY,
    name TEXT,
    latitude REAL,
    longitude REAL
);""")

conn.commit()

cursor.execute("""CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    email TEXT,
    age INTEGER,
    gender TEXT,
    state TEXT,
    street_address TEXT,
    postal_code TEXT,
    city TEXT,
    country TEXT,
    latitude REAL,
    longitude REAL,
    traffic_source TEXT,
    created_at TIMESTAMP
);""")

conn.commit()

cursor.execute("""CREATE TABLE events (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    sequence_number INTEGER,
    session_id TEXT,
    created_at TIMESTAMP,
    ip_address TEXT,
    city TEXT,
    state TEXT,
    postal_code TEXT,
    browser TEXT,
    traffic_source TEXT,
    uri TEXT,
    event_type TEXT,
    FOREIGN KEY (user_id) REFERENCES users(id)
);""")

conn.commit()

cursor.execute("""CREATE TABLE products (
    id INTEGER PRIMARY KEY,
    cost REAL,
    category TEXT,
    name TEXT,
    brand TEXT,
    retail_price REAL,
    department TEXT,
    sku TEXT,
    distribution_center_id INTEGER,
    FOREIGN KEY (distribution_center_id) REFERENCES distribution_centers(id)
);""")

conn.commit()

cursor.execute("""CREATE TABLE inventory_items (
    id INTEGER PRIMARY KEY,
    product_id INTEGER,
    created_at TIMESTAMP,
    sold_at TIMESTAMP,
    cost REAL,
    product_category TEXT,
    product_name TEXT,
    product_brand TEXT,
    product_retail_price REAL,
    product_department TEXT,
    product_sku TEXT,
    product_distribution_center_id INTEGER,
    FOREIGN KEY (product_id) REFERENCES products(id),
    FOREIGN KEY (product_distribution_center_id) REFERENCES distribution_centers(id)
);""")

conn.commit()

cursor.execute("""CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    status TEXT,
    gender TEXT,
    created_at TIMESTAMP,
    returned_at TIMESTAMP,
    shipped_at TIMESTAMP,
    delivered_at TIMESTAMP,
    num_of_item INTEGER,
    FOREIGN KEY (user_id) REFERENCES users(id)
);""")

conn.commit()

cursor.execute("""CREATE TABLE order_items (
    id INTEGER PRIMARY KEY,
    order_id INTEGER,
    user_id INTEGER,
    product_id INTEGER,
    inventory_item_id INTEGER,
    status TEXT,
    created_at TIMESTAMP,
    shipped_at TIMESTAMP,
    delivered_at TIMESTAMP,
    returned_at TIMESTAMP,
    sale_price REAL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (user_id) REFERENCES users(id),
    FOREIGN KEY (product_id) REFERENCES products(id),
    FOREIGN KEY (inventory_item_id) REFERENCES inventory_items(id)
);""")

conn.commit()

In [6]:
distribution_centers = pd.read_csv("distribution_centers.csv")
events = pd.read_csv("events.csv")
inventory_items = pd.read_csv("inventory_items.csv")
order_items = pd.read_csv("order_items.csv")
orders = pd.read_csv("orders.csv")
products = pd.read_csv("products.csv")
users = pd.read_csv("users.csv")

In [20]:
# To do INSERT INTO table (column1, column2 ... columnN) VALUES (?, ?, ... ?)

def insert_dataframe(df, table_name):
    cols = ', '.join(df.columns)
    placeholders = ', '.join(['?'] * len(df.columns))
    sql = f"INSERT INTO {table_name} ({cols}) VALUES ({placeholders})"
    cursor.executemany(sql, df.values.tolist())

insert_dataframe(distribution_centers, "distribution_centers")
insert_dataframe(users, "users")
insert_dataframe(events, "events")
insert_dataframe(products, "products")
insert_dataframe(inventory_items, "inventory_items")
insert_dataframe(orders, "orders")
insert_dataframe(order_items, "order_items")

In [21]:
conn.commit()
print("All data inserted successfully with correct schema.")

All data inserted successfully with correct schema.


### EDA: Schema Overview, Descriptive Stats, Missing Values, Categorical Distributions, Additonal Queries

#### Schema Overview:
(Just to make sure everything is okay)

In [2]:
pd.read_sql_query("PRAGMA table_info(distribution_centers);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,name,TEXT,0,None,0
2,2,latitude,REAL,0,None,0
3,3,longitude,REAL,0,None,0


In [3]:
pd.read_sql_query("PRAGMA table_info(events);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,user_id,INTEGER,0,None,0
2,2,sequence_number,INTEGER,0,None,0
3,3,session_id,TEXT,0,None,0
4,4,created_at,TIMESTAMP,0,None,0
5,5,ip_address,TEXT,0,None,0
6,6,city,TEXT,0,None,0
7,7,state,TEXT,0,None,0
8,8,postal_code,TEXT,0,None,0
9,9,browser,TEXT,0,None,0


In [4]:
pd.read_sql_query("PRAGMA table_info(inventory_items);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,product_id,INTEGER,0,None,0
2,2,created_at,TIMESTAMP,0,None,0
3,3,sold_at,TIMESTAMP,0,None,0
4,4,cost,REAL,0,None,0
5,5,product_category,TEXT,0,None,0
6,6,product_name,TEXT,0,None,0
7,7,product_brand,TEXT,0,None,0
8,8,product_retail_price,REAL,0,None,0
9,9,product_department,TEXT,0,None,0


In [5]:
pd.read_sql_query("PRAGMA table_info(order_items);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,order_id,INTEGER,0,None,0
2,2,user_id,INTEGER,0,None,0
3,3,product_id,INTEGER,0,None,0
4,4,inventory_item_id,INTEGER,0,None,0
5,5,status,TEXT,0,None,0
6,6,created_at,TIMESTAMP,0,None,0
7,7,shipped_at,TIMESTAMP,0,None,0
8,8,delivered_at,TIMESTAMP,0,None,0
9,9,returned_at,TIMESTAMP,0,None,0


In [6]:
pd.read_sql_query("PRAGMA table_info(orders);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,order_id,INTEGER,0,None,1
1,1,user_id,INTEGER,0,None,0
2,2,status,TEXT,0,None,0
3,3,gender,TEXT,0,None,0
4,4,created_at,TIMESTAMP,0,None,0
5,5,returned_at,TIMESTAMP,0,None,0
6,6,shipped_at,TIMESTAMP,0,None,0
7,7,delivered_at,TIMESTAMP,0,None,0
8,8,num_of_item,INTEGER,0,None,0


In [7]:
pd.read_sql_query("PRAGMA table_info(products);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,cost,REAL,0,None,0
2,2,category,TEXT,0,None,0
3,3,name,TEXT,0,None,0
4,4,brand,TEXT,0,None,0
5,5,retail_price,REAL,0,None,0
6,6,department,TEXT,0,None,0
7,7,sku,TEXT,0,None,0
8,8,distribution_center_id,INTEGER,0,None,0


In [2]:
pd.read_sql_query("PRAGMA table_info(users);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,first_name,TEXT,0,None,0
2,2,last_name,TEXT,0,None,0
3,3,email,TEXT,0,None,0
4,4,age,INTEGER,0,None,0
5,5,gender,TEXT,0,None,0
6,6,state,TEXT,0,None,0
7,7,street_address,TEXT,0,None,0
8,8,postal_code,TEXT,0,None,0
9,9,city,TEXT,0,None,0


#### Missing Values

In [9]:
table_columns = {
    "distribution_centers": [
        "id", "name", "latitude", "longitude"
    ],
    "events": [
        "id", "user_id", "sequence_number", "session_id", "created_at",
        "ip_address", "city", "state", "postal_code", "browser",
        "traffic_source", "uri", "event_type"
    ],
    "inventory_items": [
        "id", "product_id", "created_at", "sold_at", "cost",
        "product_category", "product_name", "product_brand",
        "product_retail_price", "product_department", "product_sku",
        "product_distribution_center_id"
    ],
    "order_items": [
        "id", "order_id", "user_id", "product_id", "inventory_item_id",
        "status", "created_at", "shipped_at", "delivered_at", "returned_at"
    ],
    "orders": [
        "order_id", "user_id", "status", "gender", "created_at",
        "returned_at", "shipped_at", "delivered_at", "num_of_item"
    ],
    "products": [
        "id", "cost", "category", "name", "brand", "retail_price",
        "department", "sku", "distribution_center_id"
    ]
}

def generate_nulls_query(table, columns):
    null_checks = ",\n    ".join(
        f"SUM(CASE WHEN {col} IS NULL THEN 1 ELSE 0 END) AS missing_{col}"
        for col in columns
    )
    return f"SELECT {null_checks} FROM {table};"

for table, columns in table_columns.items():
    print(f"\n Missing value summary for '{table}':\n")
    query = generate_nulls_query(table, columns)
    result = pd.read_sql_query(query, conn)
    display(result)


 Missing value summary for 'distribution_centers':



,missing_id,missing_name,missing_latitude,missing_longitude
0,0,0,0,0



 Missing value summary for 'events':



,missing_id,missing_user_id,missing_sequence_number,missing_session_id,missing_created_at,missing_ip_address,missing_city,missing_state,missing_postal_code,missing_browser,missing_traffic_source,missing_uri,missing_event_type
0,0,1125671,0,0,0,0,23080,0,0,0,0,0,0



 Missing value summary for 'inventory_items':



...

,missing_id,missing_order_id,missing_user_id,missing_product_id,missing_inventory_item_id,missing_status,missing_created_at,missing_shipped_at,missing_delivered_at,missing_returned_at
0,0,0,0,0,0,0,0,63478,117918,163527



 Missing value summary for 'orders':



,missing_order_id,missing_user_id,missing_status,missing_gender,missing_created_at,missing_returned_at,missing_shipped_at,missing_delivered_at,missing_num_of_item
0,0,0,0,0,0,112696,43765,81342,0



 Missing value summary for 'products':



,missing_id,missing_cost,missing_category,missing_name,missing_brand,missing_retail_price,missing_department,missing_sku,missing_distribution_center_id
0,0,0,0,2,24,0,0,0,0


In [10]:
query = """
SELECT 
    SUM(CASE WHEN id IS NULL THEN 1 ELSE 0 END) AS missing_id,
    SUM(CASE WHEN first_name IS NULL THEN 1 ELSE 0 END) AS missing_first_name,
    SUM(CASE WHEN last_name IS NULL THEN 1 ELSE 0 END) AS missing_last_name,
    SUM(CASE WHEN email IS NULL THEN 1 ELSE 0 END) AS missing_email,
    SUM(CASE WHEN age IS NULL THEN 1 ELSE 0 END) AS missing_age,
    SUM(CASE WHEN gender IS NULL THEN 1 ELSE 0 END) AS missing_gender,
    SUM(CASE WHEN state IS NULL THEN 1 ELSE 0 END) AS missing_state,
    SUM(CASE WHEN street_address IS NULL THEN 1 ELSE 0 END) AS missing_street_address,
    SUM(CASE WHEN postal_code IS NULL THEN 1 ELSE 0 END) AS missing_postal_code,
    SUM(CASE WHEN city IS NULL THEN 1 ELSE 0 END) AS missing_city,
    SUM(CASE WHEN country IS NULL THEN 1 ELSE 0 END) AS missing_country,
    SUM(CASE WHEN latitude IS NULL THEN 1 ELSE 0 END) AS missing_latitude,
    SUM(CASE WHEN longitude IS NULL THEN 1 ELSE 0 END) AS missing_longitude,
    SUM(CASE WHEN traffic_source IS NULL THEN 1 ELSE 0 END) AS missing_traffic_source,
    SUM(CASE WHEN created_at IS NULL THEN 1 ELSE 0 END) AS missing_created_at
FROM users;
"""

pd.read_sql_query(query, conn)

,missing_id,missing_first_name,missing_last_name,missing_email,missing_age,missing_gender,missing_state,missing_street_address,missing_postal_code,missing_city,missing_country,missing_latitude,missing_longitude,missing_traffic_source,missing_created_at
0,0,0,0,0,0,0,0,0,0,958,0,0,0,0,0


In [11]:
query = """
SELECT COUNT(*) FROM events;
"""

pd.read_sql_query(query, conn)

,COUNT(*)
0,2431963


There are quite a few missing values, however it doesn't make sense to fill them with specific values as many are timestaps, user_ids... which we can't really fill in a coherent way. Moreover we can't either drop those rows either as we would have to get rid of a lot of data, for example in the events table 1125671 rows have missing user_ids, which a little less than half.

#### EDA for relevant tables (descriptive stats, categorical value counts, other queries)

#### Users table:

In [ ]:
query = """
SELECT gender, COUNT(*) AS count
FROM users
GROUP BY gender;
"""

pd.read_sql_query(query, conn)

,gender,count
0,F,50208
1,M,49792


Approximately even split between F/M

In [33]:
query = """
SELECT 
    COUNT(*) AS count,
    AVG(age) AS avg_age,
    MIN(age) AS min_age,
    MAX(age) AS max_age
FROM users;
"""

pd.read_sql_query(query, conn)

,count,avg_age,min_age,max_age
0,100000,41.05479,12,70


In [34]:
query = """
WITH ordered_ages AS (
    SELECT age,
           ROW_NUMBER() OVER (ORDER BY age) AS rn,
           COUNT(*) OVER () AS total_count
    FROM users
    WHERE age IS NOT NULL
)
SELECT AVG(age) AS median_age
FROM ordered_ages
WHERE rn IN ((total_count + 1) / 2, (total_count + 2) / 2);
"""

pd.read_sql_query(query, conn)

,median_age
0,41.0


In [35]:
query = """
WITH stats AS (
    SELECT 
        AVG(age) AS mean_age,
        COUNT(age) AS n
    FROM users
    WHERE age IS NOT NULL
),
variance_calc AS (
    SELECT 
        SUM((age - (SELECT mean_age FROM stats)) * (age - (SELECT mean_age FROM stats))) AS sum_sq_diff
    FROM users
    WHERE age IS NOT NULL
)
SELECT 
    SQRT(sum_sq_diff / (SELECT n FROM stats)) AS stddev_age
FROM variance_calc;
"""

pd.read_sql_query(query, conn)

,stddev_age
0,17.043829


In [ ]:
#25 percentile
query = """
WITH ordered_ages AS (
    SELECT age,
           PERCENT_RANK() OVER (ORDER BY age) AS pr
    FROM users
    WHERE age IS NOT NULL
)
SELECT age
FROM ordered_ages
WHERE pr >= 0.25
ORDER BY pr
LIMIT 1;
"""

pd.read_sql_query(query, conn)

,age
0,27


In [ ]:
#75 percentile
query = """
WITH ordered_ages AS (
    SELECT age,
           PERCENT_RANK() OVER (ORDER BY age) AS pr
    FROM users
    WHERE age IS NOT NULL
)
SELECT age
FROM ordered_ages
WHERE pr >= 0.75
ORDER BY pr
LIMIT 1;
"""

pd.read_sql_query(query, conn)

,age
0,57


In [49]:
query = """
SELECT
  CASE
    WHEN age < 18 THEN 'Under 18'
    WHEN age BETWEEN 18 AND 24 THEN '18-24'
    WHEN age BETWEEN 25 AND 34 THEN '25-34'
    WHEN age BETWEEN 35 AND 44 THEN '35-44'
    WHEN age BETWEEN 45 AND 54 THEN '45-54'
    WHEN age >= 55 THEN '55+'
    ELSE 'Unknown'
  END AS age_group,
  COUNT(*) AS user_count,
  AVG(age) AS avg_age
FROM users
GROUP BY age_group
ORDER BY age_group;
"""

pd.read_sql_query(query, conn)

,age_group,user_count,avg_age
0,18-24,11896,20.983104
1,25-34,16803,29.517110
2,35-44,16939,39.500797
3,45-54,16882,49.505983
4,55+,27330,62.480790
5,Under 18,10150,14.524532


In [39]:
query = """
SELECT country, COUNT(*) AS user_count
FROM users
GROUP BY country
ORDER BY user_count DESC
LIMIT 10;
"""

pd.read_sql_query(query, conn)

,country,user_count
0,China,34150
1,United States,22522
2,Brasil,14507
3,South Korea,5316
4,France,4700
5,United Kingdom,4561
6,Germany,4155
7,Spain,4062
8,Japan,2438
9,Australia,2146


In [40]:
query = """
SELECT city, COUNT(*) AS user_count
FROM users
GROUP BY city
ORDER BY user_count DESC
LIMIT 10;
"""

pd.read_sql_query(query, conn)

,city,user_count
0,Shanghai,2525
1,Beijing,2175
2,Seoul,1483
3,Shenzhen,1320
4,None,958
5,Dongguan,942
6,Chengdu,844
7,New York,840
8,Tokyo,759
9,Foshan,753


In [38]:
query = """
SELECT state, COUNT(*) AS user_count
FROM users
GROUP BY state
ORDER BY user_count DESC
LIMIT 10;
"""

pd.read_sql_query(query, conn)

,state,user_count
0,Guangdong,5380
1,England,4034
2,California,3704
3,Shanghai,2499
4,Texas,2468
5,Zhejiang,2125
6,Beijing,2123
7,São Paulo,2119
8,Hebei,2038
9,Jiangsu,1894


In [ ]:
query = """
SELECT
  COUNT(DISTINCT state) AS unique_states,
  COUNT(DISTINCT city) AS unique_cities,
  COUNT(DISTINCT country) AS unique_countries
FROM users;
"""

pd.read_sql_query(query, conn)

,unique_states,unique_cities,unique_countries
0,229,7883,16


In [42]:
query = """
SELECT country, ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM users), 2) AS pct_users
FROM users
GROUP BY country
ORDER BY pct_users DESC;
"""

pd.read_sql_query(query, conn)

,country,pct_users
0,China,34.15
1,United States,22.52
2,Brasil,14.51
3,South Korea,5.32
4,France,4.70
5,United Kingdom,4.56
6,Germany,4.16
7,Spain,4.06
8,Japan,2.44
9,Australia,2.15


In [44]:
query = """
SELECT COUNT(DISTINCT traffic_source) AS unique_traffic_sources FROM users;
"""

pd.read_sql_query(query, conn)

,unique_traffic_sources
0,5


In [43]:
query = """
SELECT traffic_source, COUNT(*) AS user_count,
       ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM users), 2) AS pct_users
FROM users
GROUP BY traffic_source
ORDER BY user_count DESC;
"""

pd.read_sql_query(query, conn)

,traffic_source,user_count,pct_users
0,Search,70075,70.08
1,Organic,15110,15.11
2,Facebook,5816,5.82
3,Email,4947,4.95
4,Display,4052,4.05


In [45]:
query = """
SELECT STRFTIME('%Y', created_at) AS year, COUNT(*) AS user_count
FROM users
GROUP BY year
ORDER BY year;
"""

pd.read_sql_query(query, conn)

,year,user_count
0,2019,19391
1,2020,19331
2,2021,19406
3,2022,19361
4,2023,19235
5,2024,3276


In [46]:
query = """
SELECT STRFTIME('%Y-%m', created_at) AS year_month, COUNT(*) AS user_count
FROM users
GROUP BY year_month
ORDER BY year_month;
"""

pd.read_sql_query(query, conn)

,year_month,user_count
0,2019-01,1591
1,2019-02,1441
2,2019-03,1639
3,2019-04,1608
4,2019-05,1727
...,...,...
56,2023-09,1620
57,2023-10,1644
58,2023-11,1576
59,2023-12,1635


In [ ]:
query = """
SELECT MIN(created_at) AS first_user_created, MAX(created_at) AS last_user_created FROM users;
"""

pd.read_sql_query(query, conn)

,first_user_created,last_user_created
0,2019-01-02 00:06:00+00:00,2024-01-16 19:46:14.316147+00:00


In [48]:
query = """
SELECT STRFTIME('%w', created_at) AS weekday, COUNT(*) AS user_count
FROM users
GROUP BY weekday
ORDER BY weekday;"""

pd.read_sql_query(query, conn)

,weekday,user_count
0,0,14350
1,1,14304
2,2,14418
3,3,13909
4,4,14393
5,5,14356
6,6,14270


In [50]:
query = """
SELECT city, traffic_source, COUNT(*) AS user_count
FROM users
GROUP BY city, traffic_source
ORDER BY user_count DESC
LIMIT 10;"""

pd.read_sql_query(query, conn)

,city,traffic_source,user_count
0,Shanghai,Search,1733
1,Beijing,Search,1486
2,Seoul,Search,1022
3,Shenzhen,Search,938
4,Dongguan,Search,666
5,None,Search,659
6,New York,Search,623
7,Chengdu,Search,604
8,Tokyo,Search,545
9,Tianjin,Search,537


In [51]:
query = """
SELECT STRFTIME('%Y', created_at) AS year, state, COUNT(*) AS user_count
FROM users
GROUP BY year, state
ORDER BY year, user_count DESC;"""

pd.read_sql_query(query, conn)

,year,state,user_count
0,2019,Guangdong,1046
1,2019,England,759
2,2019,California,754
3,2019,Texas,475
4,2019,Shanghai,457
...,...,...,...
1306,2024,Extremadura,1
1307,2024,Dolnośląskie,1
1308,2024,District of Columbia,1
1309,2024,Bremen,1


In [53]:
query = """
SELECT
  STRFTIME('%Y-%m', created_at) AS year_month,
  traffic_source,
  COUNT(*) AS user_count
FROM users
GROUP BY year_month, traffic_source
ORDER BY year_month, user_count DESC;
"""

pd.read_sql_query(query, conn)

,year_month,traffic_source,user_count
0,2019-01,Search,1088
1,2019-01,Organic,253
2,2019-01,Facebook,95
3,2019-01,Email,83
4,2019-01,Display,72
...,...,...,...
300,2024-01,Search,2343
301,2024-01,Organic,453
302,2024-01,Facebook,204
303,2024-01,Email,146


In [13]:

query = """
SELECT city, gender, COUNT(*) AS user_count, AVG(age) AS avg_age
FROM users
GROUP BY city, gender
ORDER BY user_count DESC
LIMIT 20;
"""

pd.read_sql_query(query, conn)

,city,gender,user_count,avg_age
0,Shanghai,F,1288,39.934006
1,Shanghai,M,1237,41.364592
2,Beijing,F,1100,41.878182
3,Beijing,M,1075,40.867907
4,Seoul,F,758,42.949868
5,Seoul,M,725,41.267586
6,Shenzhen,M,667,41.343328
7,Shenzhen,F,653,41.208270
8,Dongguan,M,512,41.455078
9,None,F,494,41.514170


In [55]:

query = """
SELECT country, gender, COUNT(*) AS user_count
FROM users
GROUP BY country, gender
ORDER BY country, user_count DESC;
"""

pd.read_sql_query(query, conn)

,country,gender,user_count
0,Australia,M,1082
1,Australia,F,1064
2,Austria,M,2
3,Belgium,M,612
4,Belgium,F,573
5,Brasil,F,7257
6,Brasil,M,7250
7,China,F,17152
8,China,M,16998
9,Colombia,F,10


In [56]:
query = """
SELECT country, traffic_source, COUNT(*) AS user_count, AVG(age) AS avg_age
FROM users
GROUP BY country, traffic_source
ORDER BY user_count DESC;
"""

pd.read_sql_query(query, conn)

,country,traffic_source,user_count,avg_age
0,China,Search,23876,40.866267
1,United States,Search,15768,41.252981
2,Brasil,Search,10147,41.293486
3,China,Organic,5141,40.723595
4,South Korea,Search,3701,41.273169
...,...,...,...,...
64,Austria,Facebook,1,36.000000
65,Austria,Search,1,64.000000
66,Colombia,Email,1,49.000000
67,Deutschland,Organic,1,68.000000


In [57]:
query = """
SELECT
  country,
  gender,
  COUNT(*) AS user_count,
  ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM users WHERE country = u.country), 2) AS pct_country
FROM users u
GROUP BY country, gender
ORDER BY country, gender;
"""

pd.read_sql_query(query, conn)

,country,gender,user_count,pct_country
0,Australia,F,1064,49.58
1,Australia,M,1082,50.42
2,Austria,M,2,100.00
3,Belgium,F,573,48.35
4,Belgium,M,612,51.65
5,Brasil,F,7257,50.02
6,Brasil,M,7250,49.98
7,China,F,17152,50.23
8,China,M,16998,49.77
9,Colombia,F,10,58.82


In [58]:
query = """
SELECT state, traffic_source, COUNT(*) AS user_count, AVG(age) AS avg_age
FROM users
GROUP BY state, traffic_source
ORDER BY user_count DESC;
"""

pd.read_sql_query(query, conn)

,state,traffic_source,user_count,avg_age
0,Guangdong,Search,3757,41.363056
1,England,Search,2836,41.187941
2,California,Search,2535,40.854043
3,Shanghai,Search,1775,40.958310
4,Texas,Search,1732,41.267321
...,...,...,...,...
1051,Wyoming,Display,1,12.000000
1052,Wyoming,Email,1,54.000000
1053,Yamaguchi,Display,1,50.000000
1054,Yamaguchi,Email,1,59.000000


#### Product table:

In [60]:
query = """
SELECT product_category, COUNT(*) AS count
FROM inventory_items
GROUP BY product_category
ORDER BY count DESC;
"""

pd.read_sql_query(query, conn)

,product_category,count
0,Intimates,36353
1,Jeans,34255
2,Tops & Tees,32285
3,Fashion Hoodies & Sweatshirts,32155
4,Swim,30870
5,Sweaters,30479
6,Shorts,29697
7,Sleep & Lounge,29648
8,Accessories,26367
9,Active,24651


In [64]:
query = """
SELECT brand, COUNT(*) AS count
FROM products
GROUP BY brand
ORDER BY count DESC
LIMIT 15;
"""

pd.read_sql_query(query, conn)

,brand,count
0,Allegra K,1034
1,Calvin Klein,497
2,Carhartt,388
3,Hanes,308
4,Volcom,292
5,Nautica,263
6,Levi's,259
7,Quiksilver,257
8,Tommy Hilfiger,251
9,Columbia,236


In [62]:
query = """
SELECT
    COUNT(*) AS product_count,
    MIN(retail_price) AS min_price,
    MAX(retail_price) AS max_price,
    ROUND(AVG(retail_price), 2) AS avg_price,
    ROUND(AVG(cost), 2) AS avg_cost,
    ROUND(AVG(retail_price - cost), 2) AS avg_margin
FROM products;
"""

pd.read_sql_query(query, conn)

,product_count,min_price,max_price,avg_price,avg_cost,avg_margin
0,29120,0.02,999.0,59.22,28.48,30.74


In [73]:
query = """
SELECT 
    department,
    COUNT(*) AS num_products,
    MIN(retail_price) AS min_price,
    MAX(retail_price) AS max_price,
    ROUND(AVG(retail_price), 2) AS avg_price
FROM products
GROUP BY department
ORDER BY num_products DESC;
"""

pd.read_sql_query(query, conn)

,department,num_products,min_price,max_price,avg_price
0,Women,15989,0.02,903.0,55.94
1,Men,13131,1.50,999.0,63.21


In [69]:
query = """
SELECT 
    category,
    COUNT(*) AS num_products,
    ROUND(AVG(cost), 2) AS avg_cost,
    ROUND(AVG(retail_price), 2) AS avg_price
FROM products
GROUP BY category
ORDER BY num_products DESC;

"""

pd.read_sql_query(query, conn)

,category,num_products,avg_cost,avg_price
0,Intimates,2363,17.92,33.71
1,Jeans,1999,52.41,97.85
2,Tops & Tees,1868,23.14,41.34
3,Fashion Hoodies & Sweatshirts,1866,27.80,53.92
4,Swim,1798,28.89,57.82
5,Sleep & Lounge,1771,24.23,49.22
6,Shorts,1765,22.92,45.77
7,Sweaters,1737,36.09,75.32
8,Accessories,1559,17.06,42.57
9,Active,1432,21.22,50.62


In [72]:
query = """
SELECT 
    brand,
    ROUND(AVG(retail_price), 2) AS avg_price,
    COUNT(*) AS num_products
FROM products
GROUP BY brand
ORDER BY num_products DESC
LIMIT 20;
"""

pd.read_sql_query(query, conn)

,brand,avg_price,num_products
0,Allegra K,14.13,1034
1,Calvin Klein,63.66,497
2,Carhartt,68.75,388
3,Hanes,19.71,308
4,Volcom,58.66,292
5,Nautica,41.38,263
6,Levi's,49.91,259
7,Quiksilver,58.04,257
8,Tommy Hilfiger,73.45,251
9,Columbia,68.47,236


In [76]:
query = """
SELECT 
    department,
    ROUND(AVG(retail_price - cost), 2) AS avg_profit,
    ROUND(AVG((retail_price - cost) / NULLIF(cost, 0)), 2) AS avg_margin_ratio
FROM products
GROUP BY department
ORDER BY avg_profit DESC;
"""

pd.read_sql_query(query, conn)

,department,avg_profit,avg_margin_ratio
0,Men,32.76,1.08
1,Women,29.08,1.10


In [79]:
query = """
SELECT 
    brand,
    COUNT(*) AS num_products,
    ROUND(AVG(retail_price - cost), 2) AS avg_profit,
    ROUND(AVG((retail_price - cost) / NULLIF(cost, 0)), 2) AS avg_margin_ratio
FROM products
GROUP BY brand
HAVING COUNT(*) > 15
ORDER BY avg_profit DESC;
"""

pd.read_sql_query(query, conn)

,brand,num_products,avg_profit,avg_margin_ratio
0,Canada Goose,18,329.68,1.35
1,The North Face,36,230.67,1.26
2,Arc'teryx,40,167.73,1.22
3,Barbour,18,165.88,1.23
4,Cashmere Boutique,21,130.05,1.24
...,...,...,...,...
379,Stance,17,5.39,0.72
380,DG Eyewear,23,5.19,1.36
381,Up2date Fashion,24,5.11,0.66
382,Braza,32,5.06,0.87


In [81]:
query = """
SELECT 
    CASE 
        WHEN retail_price < 10 THEN 'Under $10'
        WHEN retail_price BETWEEN 10 AND 20 THEN '$10 - $20'
        WHEN retail_price BETWEEN 20 AND 50 THEN '$20 - $50'
        WHEN retail_price BETWEEN 50 AND 100 THEN '$50 - $100'
        WHEN retail_price BETWEEN 100 AND 250 THEN '$100 - $250'
        ELSE 'Over $250'
    END AS price_range,
    COUNT(*) AS num_products
FROM products
GROUP BY price_range
ORDER BY COUNT(*) DESC;
"""

pd.read_sql_query(query, conn)

,price_range,num_products
0,$20 - $50,12086
1,$50 - $100,7004
2,$10 - $20,4683
3,$100 - $250,3504
4,Under $10,1345
5,Over $250,498


In [83]:
query = """
SELECT 
    category,
    ROUND(AVG(retail_price), 2) AS avg_price
FROM products
GROUP BY category
ORDER BY avg_price DESC
LIMIT 10;
"""

pd.read_sql_query(query, conn)

,category,avg_price
0,Outerwear & Coats,146.02
1,Suits & Sport Coats,126.56
2,Suits,116.16
3,Jeans,97.85
4,Blazers & Jackets,92.60
5,Clothing Sets,84.84
6,Dresses,84.20
7,Sweaters,75.32
8,Pants,59.81
9,Swim,57.82


In [84]:
query = """
SELECT 
    category,
    ROUND(AVG(retail_price), 2) AS avg_price
FROM products
GROUP BY category
ORDER BY avg_price
LIMIT 10;
"""

pd.read_sql_query(query, conn)

,category,avg_price
0,Socks & Hosiery,16.76
1,Socks,20.42
2,Leggings,27.15
3,Underwear,27.16
4,Intimates,33.71
5,Plus,38.43
6,Tops & Tees,41.34
7,Accessories,42.57
8,Jumpsuits & Rompers,45.42
9,Shorts,45.77


In [85]:
query = """
SELECT 
    id,
    name,
    brand,
    ROUND(cost, 2) AS cost,
    ROUND(retail_price, 2) AS price,
    ROUND(retail_price - cost, 2) AS profit
FROM products
WHERE cost > 0
ORDER BY profit DESC
LIMIT 10;
"""

pd.read_sql_query(query, conn)

,id,name,brand,cost,price,profit
0,24447,Darla,Alpha Industries,404.60,999.0,594.40
1,24341,Nobis Yatesy Parka,Nobis,381.90,950.0,568.10
2,23654,The North Face Apex Bionic Soft Shell Jacket -...,The North Face,363.01,903.0,539.99
3,23951,The North Face Nuptse 2 Jacket Deep Water Blue...,The North Face,367.52,903.0,535.48
4,23989,The North Face Freedom Mens Ski Pants 2013,The North Face,369.33,903.0,533.67
5,18656,JORDAN DURASHEEN SHORT MENS 404309-109,Jordan,370.23,903.0,532.77
6,24283,The North Face Nuptse 2 Jacket - Noah Green/TN...,The North Face,370.23,903.0,532.77
7,2796,ASCIS Cushion Low Socks (Pack of 3),ASICS,373.84,903.0,529.16
8,2793,adidas Women's adiFIT Slim Pant,adidas,375.65,903.0,527.35
9,8429,The North Face Women's S-XL Oso Jacket,The North Face,378.36,903.0,524.64


In [86]:
query = """
SELECT 
    category,
    ROUND(AVG(cost / NULLIF(retail_price, 0)), 2) AS avg_cost_to_price_ratio
FROM products
GROUP BY category
ORDER BY avg_cost_to_price_ratio DESC;
"""

pd.read_sql_query(query, conn)

,category,avg_cost_to_price_ratio
0,Clothing Sets,0.62
1,Suits,0.60
2,Socks,0.60
3,Leggings,0.60
4,Tops & Tees,0.56
5,Jeans,0.54
6,Pants & Capris,0.53
7,Jumpsuits & Rompers,0.53
8,Intimates,0.53
9,Swim,0.51


In [88]:
query = """
SELECT 
    department,
    COUNT(DISTINCT brand) AS num_brands,
    COUNT(*) AS num_products
FROM products
GROUP BY department
ORDER BY num_brands DESC;
"""

pd.read_sql_query(query, conn)

,department,num_brands,num_products
0,Women,1897,15989
1,Men,1330,13131


In [89]:
query = """
SELECT 
    department,
    category,
    COUNT(*) AS num_products
FROM products
GROUP BY department, category
ORDER BY department, num_products DESC;
"""

pd.read_sql_query(query, conn)

,department,category,num_products
0,Men,Jeans,1117
1,Men,Underwear,1088
2,Men,Tops & Tees,1084
3,Men,Pants,1041
4,Men,Sweaters,999
5,Men,Fashion Hoodies & Sweatshirts,971
6,Men,Sleep & Lounge,941
7,Men,Shorts,939
8,Men,Swim,906
9,Men,Socks,905


In [90]:
query = """
SELECT 
    brand,
    category,
    COUNT(*) AS num_products,
    ROUND(AVG(retail_price - cost), 2) AS avg_profit
FROM products
WHERE cost IS NOT NULL AND retail_price IS NOT NULL
GROUP BY brand, category
HAVING COUNT(*) > 5
ORDER BY avg_profit DESC
LIMIT 20;
"""

pd.read_sql_query(query, conn)

,brand,category,num_products,avg_profit
0,Canada Goose,Outerwear & Coats,12,348.26
1,Diesel,Outerwear & Coats,7,256.34
2,The North Face,Active,6,250.71
3,The North Face,Outerwear & Coats,26,244.28
4,Regency New York,Sleep & Lounge,6,211.40
5,Ben Sherman,Suits & Sport Coats,6,204.07
6,Magaschoni,Sweaters,7,198.90
7,Parker,Dresses,8,198.10
8,Diesel,Suits & Sport Coats,6,198.06
9,Cashmere Boutique,Outerwear & Coats,8,197.73


In [92]:
query = """
SELECT 
    brand,
    COUNT(*) AS num_products,
    ROUND(AVG(retail_price), 2) AS avg_price
FROM products
GROUP BY brand
HAVING COUNT(*) > 10 AND AVG(retail_price) > 100
ORDER BY avg_price DESC;
"""

pd.read_sql_query(query, conn)

,brand,num_products,avg_price
0,Canada Goose,18,581.38
1,The North Face,36,417.71
2,Parker,11,330.94
3,Arc'teryx,40,306.52
4,Barbour,18,304.61
...,...,...,...
83,Woolrich,81,102.78
84,Lilly Pulitzer,19,102.77
85,Rich & Skinny,12,101.12
86,Not Your Daughter's Jeans,89,100.53


#### Orders and Order Items table:

In [14]:
query = """
SELECT 
    DATE(created_at) AS order_date,
    COUNT(*) AS num_orders
FROM orders
GROUP BY order_date
ORDER BY order_date;
"""

pd.read_sql_query(query, conn)

,order_date,num_orders
0,2019-01-06,1
1,2019-01-07,1
2,2019-01-11,1
3,2019-01-13,1
4,2019-01-14,3
...,...,...
1813,2024-01-13,758
1814,2024-01-14,1014
1815,2024-01-15,1636
1816,2024-01-16,591


In [15]:
query = """
SELECT 
    STRFTIME('%Y', created_at) AS order_date_year,
    COUNT(*) AS num_orders
FROM orders
GROUP BY order_date_year
ORDER BY order_date_year;
"""

pd.read_sql_query(query, conn)

,order_date_year,num_orders
0,2019,2538
1,2020,8642
2,2021,16704
3,2022,29171
4,2023,59041
5,2024,9130


In [16]:
query = """
SELECT 
    STRFTIME('%Y-%m', created_at) AS order_date_year_month,
    COUNT(*) AS num_orders
FROM orders
GROUP BY order_date_year_month
ORDER BY order_date_year_month;
"""

pd.read_sql_query(query, conn)

,order_date_year_month,num_orders
0,2019-01,22
1,2019-02,38
2,2019-03,84
3,2019-04,104
4,2019-05,167
...,...,...
56,2023-09,5429
57,2023-10,6108
58,2023-11,6915
59,2023-12,8347


In [17]:

query = """
SELECT 
    status,
    COUNT(*) AS count
FROM orders
GROUP BY status
ORDER BY count DESC;
"""

pd.read_sql_query(query, conn)

,status,count
0,Shipped,37577
1,Complete,31354
2,Processing,25156
3,Cancelled,18609
4,Returned,12530


In [19]:

query = """
SELECT 
    ROUND(AVG(num_of_item), 2) AS avg_items,
    MIN(num_of_item) AS min_items,
    MAX(num_of_item) AS max_items
FROM orders;
"""

pd.read_sql_query(query, conn)

,avg_items,min_items,max_items
0,1.45,1,4


In [21]:
query = """
WITH stats AS (
    SELECT 
        AVG(num_of_item) AS mean_nb_items,
        COUNT(num_of_item) AS n
    FROM orders
    WHERE num_of_item IS NOT NULL
),
variance_calc AS (
    SELECT 
        SUM((num_of_item - (SELECT mean_nb_items FROM stats)) * (num_of_item - (SELECT mean_nb_items FROM stats))) AS sum_sq_diff
    FROM orders
    WHERE num_of_item IS NOT NULL
)
SELECT 
    SQRT(sum_sq_diff / (SELECT n FROM stats)) AS stddev_nb_items
FROM variance_calc;
"""

pd.read_sql_query(query, conn)

,stddev_nb_items
0,0.808187


In [22]:

query = """
SELECT 
    COUNT(*) AS total_orders,
    COUNT(returned_at) AS num_returns,
    ROUND(COUNT(returned_at) * 100.0 / COUNT(*), 2) AS return_rate_percent
FROM orders;
"""

pd.read_sql_query(query, conn)

,total_orders,num_returns,return_rate_percent
0,125226,12530,10.01


In [23]:

query = """
SELECT 
    ROUND(AVG(JULIANDAY(shipped_at) - JULIANDAY(created_at)), 2) AS avg_days_to_ship,
    ROUND(AVG(JULIANDAY(delivered_at) - JULIANDAY(shipped_at)), 2) AS avg_shipping_duration,
    ROUND(AVG(JULIANDAY(delivered_at) - JULIANDAY(created_at)), 2) AS avg_total_delivery_time
FROM orders
WHERE shipped_at IS NOT NULL AND delivered_at IS NOT NULL;
"""

pd.read_sql_query(query, conn)

,avg_days_to_ship,avg_shipping_duration,avg_total_delivery_time
0,1.5,2.51,4.01


In [3]:
query = """
SELECT 
    oi.product_id,
    COUNT(*) AS total_units_sold,
    ROUND(SUM(oi.sale_price), 2) AS total_revenue
FROM order_items AS oi
WHERE oi.status = 'Complete'
GROUP BY oi.product_id;
"""

pd.read_sql_query(query, conn)

,product_id,total_units_sold,total_revenue
0,2,2,139.00
1,3,1,69.50
2,4,2,216.00
3,5,3,282.00
4,6,1,132.00
...,...,...,...
22946,29116,4,407.04
22947,29117,3,324.00
22948,29118,1,110.00
22949,29119,1,48.42


In [5]:
query = """
SELECT 
    oi.product_id,
    COUNT(*) AS total_units_sold,
    ROUND(SUM(oi.sale_price), 2) AS total_revenue
FROM order_items AS oi
WHERE oi.status = 'Complete'
GROUP BY oi.product_id
HAVING total_units_sold > 5;
"""

pd.read_sql_query(query, conn)

,product_id,total_units_sold,total_revenue
0,1169,6,389.94
1,1353,6,149.82
2,1491,6,779.94
3,2062,7,279.02
4,2423,6,132.00
...,...,...,...
178,28585,6,599.70
179,28636,6,442.38
180,28710,6,119.70
181,28722,6,512.52


In [11]:

query = """
SELECT 
    product_id,
    COUNT(*) AS units_sold,
    ROUND(SUM(sale_price), 2) AS total_revenue
FROM order_items
WHERE status = 'Complete'
GROUP BY product_id
ORDER BY units_sold DESC
LIMIT 20;
"""

pd.read_sql_query(query, conn)

,product_id,units_sold,total_revenue
0,21025,8,600.00
1,19188,8,416.00
2,18296,8,480.00
3,18039,8,351.68
4,17876,8,391.68
5,17045,8,183.92
6,15303,8,192.00
7,28176,7,260.40
8,27376,7,224.00
9,27173,7,118.86


In [10]:

query = """
SELECT 
    p.name,
    p.brand,
    COUNT(*) AS total_units_sold,
    ROUND(SUM(oi.sale_price), 2) AS total_revenue
FROM order_items AS oi
JOIN products AS p ON oi.product_id = p.id
WHERE oi.status = 'Complete'
GROUP BY p.name, p.brand
ORDER BY total_units_sold DESC
LIMIT 20;
"""

pd.read_sql_query(query, conn)

,name,brand,total_units_sold,total_revenue
0,Wrangler Men's Premium Performance Cowboy Cut ...,Wrangler,21,992.70
1,Thorlo Unisex Experia Running Sock,Thorlo,13,207.87
2,HUGO BOSS Men's Long Pant,HUGO BOSS,12,870.71
3,Smartwool Men's Midweight Funnel Zip,SmartWool,12,1799.40
4,7 For All Mankind Men's Standard Classic Strai...,7 For All Mankind,11,1722.00
5,Pearl iZUMi Attack Sock 3-Pack,Pearl iZUMi,11,255.53
6,Suncloud Optics Aviator Sunglasses,Suncloud,11,387.86
7,Anita Women's Extreme Control Sport Bra,Anita,10,750.70
8,Elegant Solid Color Infinity Loop Jersey Scarf...,TrendsBlue,10,79.90
9,Intimo Men's Solid Corel Fleece Shawl Collar Robe,Intimo,10,369.90


In [13]:
query = """
SELECT 
    p.category,
    ROUND(SUM(oi.sale_price), 2) AS revenue
FROM order_items oi
JOIN products p ON oi.product_id = p.id
WHERE oi.status = 'Complete'
GROUP BY p.category
ORDER BY revenue DESC;
"""

pd.read_sql_query(query, conn)

,category,revenue
0,Outerwear & Coats,329564.11
1,Jeans,314433.31
2,Sweaters,216189.71
3,Suits & Sport Coats,163445.08
4,Swim,161576.15
5,Fashion Hoodies & Sweatshirts,159889.72
6,Sleep & Lounge,137708.17
7,Shorts,124042.40
8,Tops & Tees,122837.34
9,Active,122688.52


In [14]:
query = """
SELECT 
    p.brand,
    ROUND(SUM(oi.sale_price), 2) AS revenue
FROM order_items oi
JOIN products p ON oi.product_id = p.id
WHERE oi.status = 'Complete'
GROUP BY p.brand
ORDER BY revenue DESC;
"""

pd.read_sql_query(query, conn)

,brand,revenue
0,Diesel,54921.03
1,Calvin Klein,49151.60
2,7 For All Mankind,48633.13
3,True Religion,47705.72
4,Carhartt,42619.55
...,...,...
2508,Extenders,3.50
2509,Chasse,3.39
2510,Wayfayrer,3.00
2511,Made in USA,0.98


In [ ]:
#AOV calculation
query = """
SELECT 
    ROUND(AVG(order_total), 2) AS avg_order_value
FROM (
    SELECT 
        order_id,
        SUM(sale_price) AS order_total
    FROM order_items
    WHERE status = 'Complete'
    GROUP BY order_id
);
"""

pd.read_sql_query(query, conn)

,avg_order_value
0,86.62


In [17]:

query = """
SELECT 
    product_id,
    COUNT(*) AS num_returns
FROM order_items
WHERE returned_at IS NOT NULL
GROUP BY product_id
ORDER BY num_returns DESC
LIMIT 15;
"""

pd.read_sql_query(query, conn)

,product_id,num_returns
0,26019,5
1,25857,5
2,25699,5
3,24141,5
4,24026,5
5,22461,5
6,19858,5
7,18768,5
8,17668,5
9,17382,5


In [ ]:
#Return rate for each brand
query = """
SELECT 
    p.brand,
    COUNT(oi.id) AS total_items,
    SUM(CASE WHEN oi.returned_at IS NOT NULL THEN 1 ELSE 0 END) AS returned_items,
    ROUND(SUM(CASE WHEN oi.returned_at IS NOT NULL THEN 1.0 ELSE 0 END) * 100.0 / COUNT(oi.id), 2) AS return_rate_percent
FROM order_items oi
JOIN products p ON oi.product_id = p.id
GROUP BY p.brand
ORDER BY return_rate_percent DESC;
"""

pd.read_sql_query(query, conn)

,brand,total_items,returned_items,return_rate_percent
0,Rig'Em Right,3,2,66.67
1,FASHION BOUTIQUE,3,2,66.67
2,C & C California,3,2,66.67
3,ADAR UNIFORMS,3,2,66.67
4,Hot Sox,5,3,60.00
...,...,...,...,...
2748,2117 of Sweden,3,0,0.00
2749,1vemoon,14,0,0.00
2750,1veMoon,5,0,0.00
2751,1826,7,0,0.00


In [19]:
#Return rate for each category
query = """
SELECT 
    p.category,
    COUNT(oi.id) AS total_items,
    SUM(CASE WHEN oi.returned_at IS NOT NULL THEN 1 ELSE 0 END) AS returned_items,
    ROUND(SUM(CASE WHEN oi.returned_at IS NOT NULL THEN 1.0 ELSE 0 END) * 100.0 / COUNT(oi.id), 2) AS return_rate_percent
FROM order_items oi
JOIN products p ON oi.product_id = p.id
GROUP BY p.category
ORDER BY return_rate_percent DESC;
"""

pd.read_sql_query(query, conn)

,category,total_items,returned_items,return_rate_percent
0,Jumpsuits & Rompers,934,106,11.35
1,Suits,1024,110,10.74
2,Suits & Sport Coats,5223,555,10.63
3,Pants,7328,772,10.53
4,Active,9086,947,10.42
5,Plus,4323,448,10.36
6,Shorts,11038,1139,10.32
7,Maternity,5165,530,10.26
8,Outerwear & Coats,8968,919,10.25
9,Fashion Hoodies & Sweatshirts,11885,1217,10.24


#### Additional Queries

In [ ]:
#Nb of orders, last order date and total spent for each customer
query = """
SELECT 
    u.id AS user_id,
    COUNT(DISTINCT o.order_id) AS num_orders,
    MAX(o.created_at) AS last_order_date,
    SUM(ii.product_retail_price) AS total_spent
FROM users u
JOIN orders o ON u.id = o.user_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN inventory_items ii ON oi.inventory_item_id = ii.id
GROUP BY u.id
ORDER BY total_spent DESC
LIMIT 10
"""

result = pd.read_sql_query(query, conn)
result.head()

,user_id,num_orders,last_order_date,total_spent
0,81262,2,2023-12-06 16:28:00+00:00,1738.969982
1,46234,3,2023-12-22 15:22:00+00:00,1576.699999
2,94183,4,2024-01-15 06:24:00+00:00,1570.990002
3,54272,2,2023-07-17 11:31:00+00:00,1490.000000
4,15746,4,2023-08-17 02:29:00+00:00,1488.799999


In [23]:
query = """
SELECT 
    u.id AS user_id,
    MIN(o.created_at) AS first_order_date,
    COUNT(o.order_id) AS total_orders
FROM users u
LEFT JOIN orders o ON u.id = o.user_id
GROUP BY u.id
HAVING total_orders > 2;
"""

pd.read_sql_query(query, conn)

,user_id,first_order_date,total_orders
0,3,2023-03-10 09:13:00+00:00,4
1,12,2023-04-06 15:13:00+00:00,4
2,17,2019-06-15 12:17:00+00:00,3
3,22,2023-11-30 15:14:00+00:00,3
4,43,2022-04-21 02:27:00+00:00,3
...,...,...,...
9976,99938,2023-11-13 18:58:00+00:00,4
9977,99952,2020-05-26 10:35:00+00:00,4
9978,99969,2023-08-20 16:04:00+00:00,4
9979,99983,2022-03-14 15:06:00+00:00,4


In [25]:
query = """
SELECT 
    u.traffic_source,
    COUNT(o.order_id) AS num_orders,
    ROUND(AVG(o.num_of_item), 2) AS avg_items_per_order
FROM users u
JOIN orders o ON u.id = o.user_id
GROUP BY u.traffic_source;
"""

pd.read_sql_query(query, conn)

,traffic_source,num_orders,avg_items_per_order
0,Display,5087,1.45
1,Email,6166,1.47
2,Facebook,7242,1.45
3,Organic,18963,1.45
4,Search,87768,1.45


In [29]:
query = """
SELECT 
    category,
    ROUND(AVG(retail_price), 2) AS avg_price,
    ROUND(AVG(cost), 2) AS avg_cost,
    ROUND(AVG(retail_price - cost), 2) AS avg_margin
FROM products
GROUP BY category;
"""

pd.read_sql_query(query, conn)

,category,avg_price,avg_cost,avg_margin
0,Accessories,42.57,17.06,25.50
1,Active,50.62,21.22,29.40
2,Blazers & Jackets,92.60,35.11,57.48
3,Clothing Sets,84.84,52.51,32.34
4,Dresses,84.20,37.93,46.27
5,Fashion Hoodies & Sweatshirts,53.92,27.80,26.11
6,Intimates,33.71,17.92,15.78
7,Jeans,97.85,52.41,45.44
8,Jumpsuits & Rompers,45.42,24.15,21.27
9,Leggings,27.15,16.28,10.87


In [30]:
query = """
SELECT 
    p.name,
    SUM(oi.sale_price - p.cost) AS total_margin
FROM order_items oi
JOIN products p ON oi.product_id = p.id
GROUP BY p.name
ORDER BY total_margin DESC
LIMIT 10;
"""

pd.read_sql_query(query, conn)

,name,total_margin
0,The North Face Apex Bionic Soft Shell Jacket -...,12296.150962
1,Canada Goose Men's The Chateau Jacket,8585.209975
2,Canada Goose Women's Solaris,8460.929984
3,NIKE WOMEN'S PRO COMPRESSION SPORTS BRA *Outst...,8218.203001
4,The North Face Apex Bionic Mens Soft Shell Ski...,7246.574977
5,Canada Goose Women's Mystique,7002.749994
6,Nobis Merideth Parka,5776.470014
7,The North Face Denali Down Womens Jacket 2013,5757.527988
8,The North Face Nuptse 2 Jacket Deep Water Blue...,5354.789986
9,JORDAN DURASHEEN SHORT MENS 404309-109,4794.929984


In [31]:
query = """
SELECT 
    product_id,
    JULIANDAY(sold_at) - JULIANDAY(created_at) AS days_in_inventory
FROM inventory_items
WHERE sold_at IS NOT NULL;
"""

pd.read_sql_query(query, conn)

,product_id,days_in_inventory
0,2953,33.592361
1,4731,54.082639
2,7656,48.323611
3,25774,20.876389
4,22308,6.533333
...,...,...
181754,21923,22.511111
181755,25959,0.962500
181756,20828,4.872222
181757,10743,9.571528


In [32]:
query = """
SELECT 
    d.name AS dc_name,
    COUNT(DISTINCT oi.order_id) AS orders_fulfilled
FROM order_items oi
JOIN inventory_items ii ON oi.inventory_item_id = ii.id
JOIN distribution_centers d ON ii.product_distribution_center_id = d.id
GROUP BY dc_name;
"""

pd.read_sql_query(query, conn)

,dc_name,orders_fulfilled
0,Charleston SC,16208
1,Chicago IL,22590
2,Houston TX,21470
3,Los Angeles CA,16580
4,Memphis TN,22688
5,Mobile AL,17729
6,New Orleans LA,12802
7,Philadelphia PA,16123
8,Port Authority of New York/New Jersey NY/NJ,15648
9,Savannah GA,11577


In [33]:
query = """
SELECT 
    user_id,
    COUNT(DISTINCT session_id) AS num_sessions
FROM events
GROUP BY user_id;
"""

pd.read_sql_query(query, conn)

,user_id,num_sessions
0,NaN,500000
1,1.0,3
2,2.0,1
3,3.0,5
4,4.0,1
...,...,...
80040,99993.0,1
80041,99996.0,4
80042,99998.0,3
80043,99999.0,2


In [34]:
query = """
SELECT 
    event_type,
    COUNT(*) AS num_events
FROM events
GROUP BY event_type
ORDER BY num_events DESC;
"""

pd.read_sql_query(query, conn)

,event_type,num_events
0,product,845607
1,cart,595994
2,department,595323
3,purchase,181759
4,cancel,125568
5,home,87712


In [35]:
query = """
SELECT 
    COUNT(*) * 1.0 / (SELECT COUNT(DISTINCT user_id) FROM events) AS bounce_rate
FROM (
    SELECT user_id
    FROM events
    GROUP BY user_id
    HAVING COUNT(DISTINCT session_id) = 1
);
"""

pd.read_sql_query(query, conn)

,bounce_rate
0,0.43721


### RFM

In [ ]:
#To figure out the cutoff date, we need the latest delivery date
query = """
SELECT 
    MAX(delivered_at)
FROM orders;
"""

pd.read_sql_query(query, conn)

,MAX(delivered_at)
0,2024-01-24 22:14:42.220676+00:00


So as a cut off date we will use 01/24/2024

In [67]:
# In this original version we do a left join so we take all the users in our user database, even those who haven't ordered yet
query = """
WITH order_data AS (
    SELECT
        o.user_id,
        o.order_id,
        MAX(o.created_at) AS last_order_date,
        COUNT(DISTINCT o.order_id) AS frequency,
        SUM(oi.sale_price) AS monetary_value
    FROM orders o
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.status = 'Complete'
    GROUP BY o.user_id
),

rfm_base AS (
    SELECT
        u.id AS user_id,
        o.last_order_date,
        o.frequency,
        o.monetary_value,
        DATE('2024-01-24') AS snapshot_date,
        JULIANDAY('2024-01-24') - JULIANDAY(o.last_order_date) AS recency
    FROM users u
    LEFT JOIN order_data o ON u.id = o.user_id
)

SELECT
    user_id,
    COALESCE(recency, 0) AS recency,
    last_order_date,
    COALESCE(frequency, 0) AS frequency,
    COALESCE(monetary_value, 0) AS monetary
FROM rfm_base;
"""

pd.read_sql_query(query, conn)

,user_id,recency,last_order_date,frequency,monetary
0,1,0.000000,None,0,0.000000
1,2,702.559722,2022-02-20 10:34:00+00:00,1,22.000000
2,3,296.615972,2023-04-02 09:13:00+00:00,1,123.470001
3,4,462.603472,2022-10-18 09:31:00+00:00,1,148.000000
4,5,0.000000,None,0,0.000000
...,...,...,...,...,...
99995,99996,0.000000,None,0,0.000000
99996,99997,0.000000,None,0,0.000000
99997,99998,0.000000,None,0,0.000000
99998,99999,0.000000,None,0,0.000000


We save the complete fixed RFM table in a view

In [73]:
cursor.execute("""
CREATE VIEW rfm_view_original AS
WITH order_data AS (
    SELECT
        o.user_id,
        o.order_id,
        MAX(o.created_at) AS last_order_date,
        COUNT(DISTINCT o.order_id) AS frequency,
        SUM(oi.sale_price) AS monetary_value
    FROM orders o
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.status = 'Complete'
    GROUP BY o.user_id
),

rfm_base AS (
    SELECT
        u.id AS user_id,
        o.last_order_date,
        o.frequency,
        o.monetary_value,
        DATE('2024-01-24') AS snapshot_date,
        JULIANDAY('2024-01-24') - JULIANDAY(o.last_order_date) AS recency
    FROM users u
    LEFT JOIN order_data o ON u.id = o.user_id
    WHERE o.frequency > 0 -- we make sure that the user has had at least 1 order
)

SELECT
    user_id,
    COALESCE(recency, 0) AS recency,
    last_order_date,
    COALESCE(frequency, 0) AS frequency,
    COALESCE(monetary_value, 0) AS monetary
FROM rfm_base;
""")

conn.commit()

In [74]:
query = """
SELECT * FROM rfm_view_original LIMIT 10;

"""

pd.read_sql_query(query, conn)

,user_id,recency,last_order_date,frequency,monetary
0,2,702.559722,2022-02-20 10:34:00+00:00,1,22.000000
1,3,296.615972,2023-04-02 09:13:00+00:00,1,123.470001
2,4,462.603472,2022-10-18 09:31:00+00:00,1,148.000000
3,17,1181.488194,2020-10-29 12:17:00+00:00,2,726.539986
4,18,203.667361,2023-07-04 07:59:00+00:00,1,25.000000
5,24,65.812500,2023-11-19 04:30:00+00:00,1,39.990002
6,29,535.229861,2022-08-06 18:29:00+00:00,1,44.000000
7,34,229.516667,2023-06-08 11:36:00+00:00,1,36.000000
8,39,515.991667,2022-08-26 00:12:00+00:00,1,18.000000
9,42,90.311111,2023-10-25 16:32:00+00:00,1,7.870000


We can add RFM scores and segment names and then save this new table as a view too

(We use CASE logic to define the scores as the NTILES were acting incorrectly)

In [ ]:
'''cursor.execute("""DROP VIEW IF EXISTS rfm_view_scores_segments;""")

conn.commit()'''

In [96]:
cursor.execute("""
CREATE VIEW rfm_view_scores_segments AS
WITH order_data AS (
    SELECT
        o.user_id,
        o.order_id,
        MAX(o.created_at) AS last_order_date,
        COUNT(DISTINCT o.order_id) AS frequency,
        SUM(oi.sale_price) AS monetary_value
    FROM orders o
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.status = 'Complete'
    GROUP BY o.user_id
),

rfm_base AS (
    SELECT
        u.id AS user_id,
        o.last_order_date,
        o.frequency,
        o.monetary_value,
        DATE('2024-01-24') AS snapshot_date,
        JULIANDAY('2024-01-24') - JULIANDAY(o.last_order_date) AS recency
    FROM users u
    LEFT JOIN order_data o ON u.id = o.user_id
    WHERE o.frequency > 0 -- we make sure that the user has had at least 1 order
), 

rfm_ranked AS (
    SELECT
        user_id,
        recency,
        frequency,
        monetary_value,
        CASE
            WHEN recency <= 90 THEN 5
            WHEN recency <= 180 THEN 4
            WHEN recency <= 360 THEN 3
            WHEN recency <= 720 THEN 2
            ELSE 1
        END AS recency_score,
        CASE
            WHEN frequency = 4 THEN 5
            WHEN frequency = 3 THEN 4
            WHEN frequency = 2 THEN 3
            WHEN frequency = 1 THEN 2
            ELSE 1
        END AS frequency_score,
        CASE
            WHEN monetary_value >= 400 THEN 5
            WHEN monetary_value >= 200 THEN 4
            WHEN monetary_value >= 100 THEN 3
            WHEN monetary_value >= 50 THEN 2
            ELSE 1
        END AS monetary_score
    FROM rfm_base
)

SELECT
    user_id,
    recency,
    frequency,
    monetary_value,
    recency_score,
    frequency_score,
    monetary_score,
    CAST(recency_score AS TEXT) || CAST(frequency_score AS TEXT) || CAST(monetary_score AS TEXT) AS rfm_score,
    CASE
        WHEN (recency_score = 5 AND frequency_score = 5 AND monetary_score = 5)
         OR (recency_score >= 4 AND frequency_score >= 4 AND monetary_score >= 4)
        THEN 'Champions'
        WHEN frequency_score >= 4 AND recency_score >= 3 THEN 'Loyal Customers'
        WHEN monetary_score = 5 AND frequency_score >= 4 THEN 'Big Spenders'
        WHEN recency_score = 5 AND frequency_score <= 2 THEN 'Recent Customers'
        WHEN recency_score <= 2 AND frequency_score >= 3 THEN 'At Risk'
        WHEN recency_score = 3 AND frequency_score BETWEEN 2 AND 3 THEN 'Need Attention'
        WHEN (recency_score = 2 OR recency_score = 3) AND frequency_score <= 2 THEN 'About to Sleep'
        WHEN recency_score = 1 AND (frequency_score = 4 OR monetary_score = 5) THEN 'Can’t Lose Them'
        WHEN recency_score BETWEEN 1 AND 2 AND frequency_score <= 2 THEN 'Hibernating'
        WHEN recency_score = 1 AND frequency_score = 1 AND monetary_score = 1 THEN 'Lost'
    ELSE 'Others'
END AS segment
FROM rfm_ranked
ORDER BY rfm_score DESC;
""")

conn.commit()

In [100]:
query = """
SELECT * FROM rfm_view_scores_segments LIMIT 10;

"""

pd.read_sql_query(query, conn)

,user_id,recency,frequency,monetary_value,recency_score,frequency_score,monetary_score,rfm_score,segment
0,52849,8.243056,4,440.859999,5,5,5,555,Champions
1,49167,59.652778,4,365.239998,5,5,4,554,Champions
2,49942,34.829167,4,242.990000,5,5,4,554,Champions
3,51001,44.490972,4,252.240004,5,5,4,554,Champions
4,54072,15.428472,4,333.340004,5,5,4,554,Champions
5,76852,8.202399,4,379.449999,5,5,4,554,Champions
6,95822,78.271528,4,355.409998,5,5,4,554,Champions
7,1163,6.560374,3,499.279993,5,4,5,545,Champions
8,5945,68.476389,3,461.500000,5,4,5,545,Champions
9,10851,13.234028,3,546.000011,5,4,5,545,Champions


Now we create a new table by aggregating with the users table for the looker dashboard

In [101]:
cursor.execute("""
CREATE TABLE rfm_customer_segments AS
WITH order_data AS (
    SELECT
        o.user_id,
        o.order_id,
        MAX(o.created_at) AS last_order_date,
        COUNT(DISTINCT o.order_id) AS frequency,
        SUM(oi.sale_price) AS monetary_value
    FROM orders o
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.status = 'Complete'
    GROUP BY o.user_id
),

rfm_base AS (
    SELECT
        u.id AS user_id,
        o.last_order_date,
        o.frequency,
        o.monetary_value,
        DATE('2024-01-24') AS snapshot_date,
        JULIANDAY('2024-01-24') - JULIANDAY(o.last_order_date) AS recency
    FROM users u
    LEFT JOIN order_data o ON u.id = o.user_id
    WHERE o.frequency > 0 -- we make sure that the user has had at least 1 order
), 

rfm_ranked AS (
    SELECT
        user_id,
        recency,
        frequency,
        monetary_value,
        CASE
            WHEN recency <= 90 THEN 5
            WHEN recency <= 180 THEN 4
            WHEN recency <= 360 THEN 3
            WHEN recency <= 720 THEN 2
            ELSE 1
        END AS recency_score,
        CASE
            WHEN frequency = 4 THEN 5
            WHEN frequency = 3 THEN 4
            WHEN frequency = 2 THEN 3
            WHEN frequency = 1 THEN 2
            ELSE 1
        END AS frequency_score,
        CASE
            WHEN monetary_value >= 400 THEN 5
            WHEN monetary_value >= 200 THEN 4
            WHEN monetary_value >= 100 THEN 3
            WHEN monetary_value >= 50 THEN 2
            ELSE 1
        END AS monetary_score
    FROM rfm_base
), 
               
final_rfm AS (
SELECT
    user_id,
    recency,
    frequency,
    monetary_value,
    recency_score,
    frequency_score,
    monetary_score,
    CAST(recency_score AS TEXT) || CAST(frequency_score AS TEXT) || CAST(monetary_score AS TEXT) AS rfm_score,
    CASE
        WHEN (recency_score = 5 AND frequency_score = 5 AND monetary_score = 5)
         OR (recency_score >= 4 AND frequency_score >= 4 AND monetary_score >= 4)
        THEN 'Champions'
        WHEN frequency_score >= 4 AND recency_score >= 3 THEN 'Loyal Customers'
        WHEN monetary_score = 5 AND frequency_score >= 4 THEN 'Big Spenders'
        WHEN recency_score = 5 AND frequency_score <= 2 THEN 'Recent Customers'
        WHEN recency_score <= 2 AND frequency_score >= 3 THEN 'At Risk'
        WHEN recency_score = 3 AND frequency_score BETWEEN 2 AND 3 THEN 'Need Attention'
        WHEN (recency_score = 2 OR recency_score = 3) AND frequency_score <= 2 THEN 'About to Sleep'
        WHEN recency_score = 1 AND (frequency_score = 4 OR monetary_score = 5) THEN 'Can’t Lose Them'
        WHEN recency_score BETWEEN 1 AND 2 AND frequency_score <= 2 THEN 'Hibernating'
        WHEN recency_score = 1 AND frequency_score = 1 AND monetary_score = 1 THEN 'Lost'
    ELSE 'Others'
END AS segment
FROM rfm_ranked
)
               
SELECT
    rfm.*,
    u.age,
    u.gender,
    u.country,
    u.city,
    u.state, 
    u.created_at          
FROM final_rfm rfm
JOIN users u ON rfm.user_id = u.id;
""")

conn.commit()

In [112]:
query = """
SELECT * FROM rfm_customer_segments;

"""

df = pd.read_sql_query(query, conn)
df

,user_id,recency,frequency,monetary_value,recency_score,frequency_score,monetary_score,rfm_score,segment,age,gender,country,city,state,created_at
0,2,702.559722,1,22.000000,2,2,1,221,About to Sleep,65,M,Brasil,Franca,São Paulo,2020-10-19 10:34:00+00:00
1,3,296.615972,1,123.470001,3,2,3,323,Need Attention,16,M,United States,Hallandale Beach,Florida,2023-03-09 09:13:00+00:00
2,4,462.603472,1,148.000000,2,2,3,223,About to Sleep,12,F,United States,Mission Bend,Texas,2019-09-14 09:31:00+00:00
3,17,1181.488194,2,726.539986,1,3,5,135,At Risk,29,M,United Kingdom,High Wycombe,England,2019-04-27 12:17:00+00:00
4,18,203.667361,1,25.000000,3,2,1,321,Need Attention,17,M,United States,Bend,Oregon,2023-06-25 07:59:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27698,99969,73.330556,2,150.940002,5,3,3,533,Others,55,M,China,Shanghai,Shanghai,2023-07-19 16:04:00+00:00
27699,99970,11.809601,1,9.990000,5,2,1,521,Recent Customers,27,F,United States,Santa Clara,California,2024-01-11 04:34:10.465974+00:00
27700,99980,675.890972,1,134.989998,2,2,3,223,About to Sleep,17,M,United States,Bradenton,Florida,2020-12-08 02:37:00+00:00
27701,99983,151.370833,1,12.990000,4,2,1,421,Others,40,M,China,Shanghai,Beijing,2022-02-27 15:06:00+00:00


In [113]:
df.to_csv('rfm_customer_segments.csv', index=False)

### Cohort Analysis

Retention analysis by signup month

In [106]:
query = """
WITH user_cohorts AS (
    SELECT
        id AS user_id,
        strftime('%Y-%m', created_at) AS cohort_month
    FROM users
),

order_months AS (
    SELECT
        user_id,
        strftime('%Y-%m', created_at) AS order_month
    FROM orders
    WHERE status = 'Complete'
),

cohort_orders AS (
    SELECT
        uc.cohort_month,
        om.order_month,
        COUNT(DISTINCT om.user_id) AS active_users
    FROM user_cohorts uc
    JOIN order_months om ON uc.user_id = om.user_id
    GROUP BY uc.cohort_month, om.order_month
),

cohort_with_period AS (
    SELECT
        cohort_month,
        order_month,
        active_users,
        CAST(
            (strftime('%Y', order_month || '-01') - strftime('%Y', cohort_month || '-01')) * 12
            + (strftime('%m', order_month || '-01') - strftime('%m', cohort_month || '-01'))
        AS INTEGER) AS months_since_signup
    FROM cohort_orders
)

SELECT
    cohort_month,
    months_since_signup,
    active_users
FROM cohort_with_period
ORDER BY cohort_month, months_since_signup;
"""

pd.read_sql_query(query, conn)

,cohort_month,months_since_signup,active_users
0,2019-01,0,5
1,2019-01,1,5
2,2019-01,2,6
3,2019-01,3,10
4,2019-01,4,11
...,...,...,...
1886,2023-11,1,236
1887,2023-11,2,126
1888,2023-12,0,216
1889,2023-12,1,259


### Churn Detection

Customers who haven't purchased in over 90 days

In [104]:
query = """
WITH last_order AS (
    SELECT
        user_id,
        MAX(created_at) AS last_order_date
    FROM orders
    GROUP BY user_id
),

churned_users AS (
    SELECT
        u.id AS user_id,
        u.created_at AS signup_date,
        lo.last_order_date,
        DATE('2024-01-24') AS snapshot_date,
        CAST(julianday('2024-01-24') - julianday(lo.last_order_date) AS INTEGER) AS days_since_last_order
    FROM users u
    LEFT JOIN last_order lo ON u.id = lo.user_id
    WHERE lo.last_order_date IS NOT NULL
      AND CAST(julianday('2024-01-24') - julianday(lo.last_order_date) AS INTEGER) > 90
)

SELECT * FROM churned_users;
"""

pd.read_sql_query(query, conn)

,user_id,signup_date,last_order_date,snapshot_date,days_since_last_order
0,1,2022-03-23 12:55:00+00:00,2022-07-18 12:55:00+00:00,2024-01-24,554
1,2,2020-10-19 10:34:00+00:00,2022-02-20 10:34:00+00:00,2024-01-24,702
2,3,2023-03-09 09:13:00+00:00,2023-08-08 09:13:00+00:00,2024-01-24,168
3,4,2019-09-14 09:31:00+00:00,2022-10-18 09:31:00+00:00,2024-01-24,462
4,5,2020-10-02 10:03:00+00:00,2022-10-20 10:03:00+00:00,2024-01-24,460
...,...,...,...,...,...
59728,99990,2020-07-29 08:26:00+00:00,2020-08-30 08:26:00+00:00,2024-01-24,1241
59729,99992,2021-12-14 14:25:00+00:00,2022-08-06 14:25:00+00:00,2024-01-24,535
59730,99996,2019-12-30 02:00:00+00:00,2023-03-13 02:00:00+00:00,2024-01-24,316
59731,99998,2022-10-31 07:27:00+00:00,2023-07-22 07:27:00+00:00,2024-01-24,185


### BigQuery 

BigQuery equivalent of important SQL queries for Looker

RFM query:

In [108]:
'''CREATE OR REPLACE TABLE ecommerce.rfm_customer_segments AS

WITH order_data AS (
    SELECT
        o.user_id,
        MAX(o.created_at) AS last_order_date,
        COUNT(DISTINCT o.order_id) AS frequency,
        SUM(oi.sale_price) AS monetary_value
    FROM `bigquery-public-data.thelook_ecommerce.orders` o
    JOIN `bigquery-public-data.thelook_ecommerce.order_items` oi ON o.order_id = oi.order_id
    WHERE o.status = 'Complete'
    GROUP BY o.user_id
),

rfm_base AS (
    SELECT
        u.id AS user_id,
        o.last_order_date,
        o.frequency,
        o.monetary_value,
        DATE('2024-01-24') AS snapshot_date,
        DATE_DIFF(DATE('2024-01-24'), DATE(o.last_order_date), DAY) AS recency
    FROM `bigquery-public-data.thelook_ecommerce.users` u
    LEFT JOIN order_data o ON u.id = o.user_id
    WHERE o.frequency IS NOT NULL AND o.frequency > 0
),

rfm_ranked AS (
    SELECT
        user_id,
        recency,
        frequency,
        monetary_value,
        CASE
            WHEN recency <= 90 THEN 5
            WHEN recency <= 180 THEN 4
            WHEN recency <= 360 THEN 3
            WHEN recency <= 720 THEN 2
            ELSE 1
        END AS recency_score,
        CASE
            WHEN frequency = 4 THEN 5
            WHEN frequency = 3 THEN 4
            WHEN frequency = 2 THEN 3
            WHEN frequency = 1 THEN 2
            ELSE 1
        END AS frequency_score,
        CASE
            WHEN monetary_value >= 400 THEN 5
            WHEN monetary_value >= 200 THEN 4
            WHEN monetary_value >= 100 THEN 3
            WHEN monetary_value >= 50 THEN 2
            ELSE 1
        END AS monetary_score
    FROM rfm_base
),

final_rfm AS (
    SELECT
        user_id,
        recency,
        frequency,
        monetary_value,
        recency_score,
        frequency_score,
        monetary_score,
        CONCAT(CAST(recency_score AS STRING), CAST(frequency_score AS STRING), CAST(monetary_score AS STRING)) AS rfm_score,
        CASE
            WHEN (recency_score = 5 AND frequency_score = 5 AND monetary_score = 5)
              OR (recency_score >= 4 AND frequency_score >= 4 AND monetary_score >= 4) THEN 'Champions'
            WHEN frequency_score >= 4 AND recency_score >= 3 THEN 'Loyal Customers'
            WHEN monetary_score = 5 AND frequency_score >= 4 THEN 'Big Spenders'
            WHEN recency_score = 5 AND frequency_score <= 2 THEN 'Recent Customers'
            WHEN recency_score <= 2 AND frequency_score >= 3 THEN 'At Risk'
            WHEN recency_score = 3 AND frequency_score BETWEEN 2 AND 3 THEN 'Need Attention'
            WHEN (recency_score = 2 OR recency_score = 3) AND frequency_score <= 2 THEN 'About to Sleep'
            WHEN recency_score = 1 AND (frequency_score = 4 OR monetary_score = 5) THEN 'Can’t Lose Them'
            WHEN recency_score BETWEEN 1 AND 2 AND frequency_score <= 2 THEN 'Hibernating'
            WHEN recency_score = 1 AND frequency_score = 1 AND monetary_score = 1 THEN 'Lost'
            ELSE 'Others'
        END AS segment
    FROM rfm_ranked
)

SELECT
    rfm.*,
    u.age,
    u.gender,
    u.country,
    u.city,
    u.state,
    u.created_at
FROM final_rfm rfm
JOIN `bigquery-public-data.thelook_ecommerce.users` u ON rfm.user_id = u.id;

'''

"CREATE OR REPLACE TABLE ecommerce.rfm_customer_segments AS\n\nWITH order_data AS (\n    SELECT\n        o.user_id,\n        MAX(o.created_at) AS last_order_date,\n        COUNT(DISTINCT o.order_id) AS frequency,\n        SUM(oi.sale_price) AS monetary_value\n    FROM `bigquery-public-data.thelook_ecommerce.orders` o\n    JOIN `bigquery-public-data.thelook_ecommerce.order_items` oi ON o.order_id = oi.order_id\n    WHERE o.status = 'Complete'\n    GROUP BY o.user_id\n),\n\nrfm_base AS (\n    SELECT\n        u.id AS user_id,\n        o.last_order_date,\n        o.frequency,\n        o.monetary_value,\n        DATE('2024-01-24') AS snapshot_date,\n        DATE_DIFF(DATE('2024-01-24'), DATE(o.last_order_date), DAY) AS recency\n    FROM `bigquery-public-data.thelook_ecommerce.users` u\n    LEFT JOIN order_data o ON u.id = o.user_id\n    WHERE o.frequency IS NOT NULL AND o.frequency > 0\n),\n\nrfm_ranked AS (\n    SELECT\n        user_id,\n        recency,\n        frequency,\n        moneta

Other relevant queries

Clustering

Do using BigQuery ML, using features like RFM scores and segments, AOV, category preference, session behavior...